In [26]:
# default_exp convert_to_zarr

# Notebook which converts per region netCDF files to Zarr files to make them more efficient when indexing

### uses pangeo_small environment

In [10]:
#test_ignore
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [9]:
#export
import xarray as xr
import zarr
from joblib import Parallel, delayed
import pandas as pd
import os

## Set the parameters here
### Ensure all State and Regions you want to transform are specified here. 

In [1]:
#test_ignore
data_root = '/media/scottcha/E1/Data/OAPMLData/'

interpolation = 1

#currently only have Washington regions and one season specified for the tutorial
#uncomment regions and seasons if doing a larger transform
regions = {#'Utah': ['Abajos', 'Logan', 'Moab', 'Ogden', 'Provo', 
           #'Salt Lake', 'Skyline', 'Uintas'],  
           'Colorado': ['Grand Mesa Zone', 'Sangre de Cristo Range', 'Steamboat Zone', 'Front Range Zone',
           'Vail Summit Zone', 'Sawatch Zone', 'Aspen Zone', 
           'North San Juan Mountains', 'South San Juan Mountains', 'Gunnison Zone'],
           #'Washington': ['Mt Hood', 'Olympics', 'Snoqualmie Pass', 'Stevens Pass',
           #'WA Cascades East, Central', 'WA Cascades East, North', 'WA Cascades East, South',
           #'WA Cascades West, Central', 'WA Cascades West, Mt Baker', 'WA Cascades West, South'
           #]
           }
seasons = ['15-16', '16-17', '17-18', '18-19']

In [2]:
#export
class ConvertToZarr:
    """
    Class which encapsulates the logic to convert a set of filtered netCDF files to Zarr
    """
    
    def __init__(self, seasons, regions, data_root, interpolate=1):
        """
        Initialize the class
        
        Keyword Arguments
        seasons: list of season values to process
        regions: dictonary of Key: State and Value: List of Regions to process for that state
        data_root: the root path of the data folders which contains the 3.GFSFiltered1xInterpolation
        interpolate: the amount of interpolation applied in in the previous ParseGFS notebook (used for finding the correct input/output paths)
        """
        self.processed_path = data_root + '/3.GFSFiltered'+ str(interpolate) + 'xInterpolation/'
        self.zarr_base_path = data_root + '/4.GFSFiltered'+ str(interpolate) + 'xInterpolationZarr/'
        
        self.seasons = seasons
        self.regions = regions
        self.data_root = data_root
        
        if not os.path.exists(self.zarr_base_path):
            os.makedirs(self.zarr_base_path)
    
    def compute_region(self, region_name, season, state):
        """
        Calculates the zarr conversion for a specific region, season and state and indexes it for efficient lookup 
        
        Keyword Arguments
        region_name: name of the region to process
        season: season to process
        state: state to process (region must be a part of the state)
        """
        first = True
        base_path = self.processed_path + season + '/' + '/Region_' + region_name 
        zarr_path = self.zarr_base_path + season + '/' + state + '/Region_' + region_name + '.zarr'
        
        #TODO: refactor these to be shared code as logic also exists in ParseGFS
        p = 181
        if season in ['15-16', '19-20']:
            p = 182 #leap years

        snow_start_date = '2015-11-01'
        if season == '16-17':
            snow_start_date = '2016-11-01'
        if season == '17-18':
            snow_start_date = '2017-11-01'
        if season == '18-19':
            snow_start_date = '2018-11-01'
        if season == '19-20':
            snow_start_date = '2019-11-01'

        date_values_pd = pd.date_range(snow_start_date, periods=p, freq="D")
        try:
            with xr.open_zarr(zarr_path) as z:
                if z.time.values[-1] == date_values_pd[-1]:
                    print(' already exists: ' + region_name + ' ' + season + ' ' + state)
                    z.close()
                    return
                else:
                    #already exists but incomplete
                    date_values_pd = [pd.Timestamp(v) for v in date_values_pd.values.astype('datetime64[ns]') if v not in z.time.values]
                    print(' some exist but have to complete ' + str(len(date_values_pd)))
                    first = False
        except ValueError as err:
            #ignore as it doesn't exist yet
            pass
        
        #sometimes vars get added, filter to only the list of vars in the first dataset for that region
        #TODO: handle the case where the first dataset has more vars than subsequent ones
        final_vars = None
        for d in date_values_pd:

            path =  base_path + '_' + d.strftime('%Y%m%d') + '.nc'
            print('On ' + str(path.split('/')[-1]))

            try:
                ds = xr.open_dataset(path, chunks={'latitude':1, 'longitude':1})
            except OSError as err:
                print(' missing file: ' + path)
                continue

            ds = ds.to_array(name='vars').chunk({'time':1, 'latitude':1, 'longitude':1, 'variable':-1}).to_dataset()

            try:

                if first:
                    final_vars = list(ds.variable.values)
                    ds.to_zarr(zarr_path, consolidated=True)
                    first=False
                else:
                    assert(final_vars is not None)
                    ds = ds.sel(variable=ds.variable.isin(final_vars))
                    ds.to_zarr(zarr_path, consolidated=True, append_dim='time')
            except ValueError as err:
                print('Value Error ' + format(err) + ' on ' + zarr_path )                
                return

    def process_tuple(self, t): 
        """
        Entry method to call compute_region with a tuple
        Basically a helper for executing with joblib parallel
        
        Keyword Arguments
        t: the tuple containing the region, season and state
        """
        self.compute_region(t[0], t[1], t[2])
    
    def make_list(self):
        """
        Helper method to make the list of values to process
        """
        to_process = []
        for s in self.seasons:
            for state in self.regions.keys():           
                for r in self.regions[state]:
                    to_process.append((r,s,state))
        return to_process
    
    def convert_local(self, jobs=15):
        l = self.make_list()
    
        #one state & season takes about 6 hours with 15 cores on my machine
        Parallel(n_jobs=jobs, backend="multiprocessing")(map(delayed(self.process_tuple), l))

In [3]:
#tmppath =  '/media/scottcha/E1/Data/OAPMLData/3.GFSFiltered1xInterpolation/19-20/Region_Mt Hood_20191120.nc'
#ds = xr.open_dataset(tmppath, chunks={'latitude':1, 'longitude':1})
#ds = ds.to_array(name='vars').chunk({'time':1, 'latitude':1, 'longitude':1, 'variable':-1}).to_dataset()

In [4]:
#final_vars = list(ds.variable.values)

In [5]:
#tmp_ds = ds.sel(variable=ds.variable.isin(final_vars))

In [6]:
#ds

In [7]:
#tmp_ds

In [11]:
#test_ignore
ctz = ConvertToZarr(seasons, regions, data_root)

In [12]:
#test_ignore
ctz.convert_local()

On Region_Grand Mesa Zone_20151101.nc
On Region_Sangre de Cristo Range_20151101.ncOn Region_Vail Summit Zone_20151101.ncOn Region_Aspen Zone_20151101.ncOn Region_Front Range Zone_20151101.nc
On Region_Steamboat Zone_20151101.nc
On Region_Sawatch Zone_20151101.ncOn Region_North San Juan Mountains_20151101.nc


On Region_Front Range Zone_20161101.ncOn Region_South San Juan Mountains_20151101.nc
On Region_Vail Summit Zone_20161101.nc



On Region_Sangre de Cristo Range_20161101.nc
On Region_Gunnison Zone_20151101.ncOn Region_Steamboat Zone_20161101.nc

On Region_Grand Mesa Zone_20161101.nc
On Region_Aspen Zone_20151102.nc
On Region_Sawatch Zone_20151102.nc
On Region_Grand Mesa Zone_20151102.nc
On Region_Vail Summit Zone_20151102.nc
On Region_Grand Mesa Zone_20161102.nc
On Region_Vail Summit Zone_20161102.nc
On Region_Gunnison Zone_20151102.nc
On Region_Aspen Zone_20151103.nc
On Region_Grand Mesa Zone_20151103.ncOn Region_Sawatch Zone_20151103.nc

On Region_Vail Summit Zone_20151103.nc
On 

On Region_Gunnison Zone_20151116.nc
On Region_Grand Mesa Zone_20161124.nc
On Region_Vail Summit Zone_20151125.nc
On Region_Vail Summit Zone_20161122.nc
On Region_Aspen Zone_20151128.nc
On Region_Grand Mesa Zone_20151128.nc
On Region_Sawatch Zone_20151128.nc
On Region_Sangre de Cristo Range_20161107.nc
On Region_Steamboat Zone_20161106.nc
On Region_North San Juan Mountains_20151109.nc
On Region_Sangre de Cristo Range_20151108.nc
On Region_Vail Summit Zone_20151126.nc
On Region_Grand Mesa Zone_20161125.nc
On Region_Aspen Zone_20151129.nc
On Region_Grand Mesa Zone_20151129.nc
On Region_Front Range Zone_20151106.nc
On Region_Vail Summit Zone_20161123.nc
On Region_Sawatch Zone_20151129.nc
On Region_Gunnison Zone_20151117.nc
On Region_Steamboat Zone_20151107.nc
On Region_Vail Summit Zone_20151127.nc
On Region_Aspen Zone_20151130.nc
On Region_Grand Mesa Zone_20161126.nc
On Region_Grand Mesa Zone_20151130.nc
On Region_South San Juan Mountains_20151109.nc
On Region_Sawatch Zone_20151130.nc
On R

On Region_Vail Summit Zone_20161213.nc
On Region_Aspen Zone_20151225.nc
On Region_Grand Mesa Zone_20151225.nc
On Region_Sawatch Zone_20151225.nc
On Region_Grand Mesa Zone_20161218.nc
On Region_South San Juan Mountains_20151116.nc
On Region_Vail Summit Zone_20151220.nc
On Region_Steamboat Zone_20161111.nc
On Region_Sangre de Cristo Range_20161113.nc
On Region_North San Juan Mountains_20151117.nc
On Region_Aspen Zone_20151226.nc
On Region_Grand Mesa Zone_20151226.nc
On Region_Gunnison Zone_20151202.nc
On Region_Vail Summit Zone_20161214.nc
On Region_Sawatch Zone_20151226.nc
On Region_Sangre de Cristo Range_20151115.nc
On Region_Grand Mesa Zone_20161219.nc
On Region_Vail Summit Zone_20151221.nc
On Region_Grand Mesa Zone_20151227.nc
On Region_Aspen Zone_20151227.nc
On Region_Front Range Zone_20151111.nc
On Region_Sawatch Zone_20151227.nc
On Region_Vail Summit Zone_20161215.nc
On Region_Steamboat Zone_20151113.nc
On Region_Grand Mesa Zone_20161220.nc
On Region_Vail Summit Zone_20151222.nc
O

On Region_Grand Mesa Zone_20160121.nc
On Region_Aspen Zone_20160121.nc
On Region_Grand Mesa Zone_20170110.nc
On Region_Sawatch Zone_20160121.nc
On Region_Vail Summit Zone_20160113.nc
On Region_Gunnison Zone_20151217.nc
On Region_Steamboat Zone_20161116.nc
On Region_Sangre de Cristo Range_20161119.nc
On Region_Vail Summit Zone_20170104.nc
On Region_Grand Mesa Zone_20160122.nc
On Region_North San Juan Mountains_20151125.nc
On Region_Aspen Zone_20160122.nc
On Region_Sawatch Zone_20160122.nc
On Region_Grand Mesa Zone_20170111.nc
On Region_Vail Summit Zone_20160114.nc
On Region_Sangre de Cristo Range_20151122.nc
On Region_Grand Mesa Zone_20160123.nc
On Region_Aspen Zone_20160123.nc
On Region_South San Juan Mountains_20151124.nc
On Region_Vail Summit Zone_20170105.nc
On Region_Sawatch Zone_20160123.nc
On Region_Gunnison Zone_20151218.nc
On Region_Grand Mesa Zone_20170112.nc
On Region_Vail Summit Zone_20160115.nc
On Region_Front Range Zone_20151116.nc
On Region_Front Range Zone_20161114.nc
On

On Region_Steamboat Zone_20151124.nc
On Region_Grand Mesa Zone_20160217.nc
On Region_Vail Summit Zone_20160206.nc
On Region_Aspen Zone_20160217.nc
On Region_Sawatch Zone_20160217.nc
On Region_Steamboat Zone_20161121.nc
On Region_Grand Mesa Zone_20170203.nc
On Region_Sangre de Cristo Range_20161125.nc
On Region_Vail Summit Zone_20170125.nc
On Region_Front Range Zone_20161118.nc
On Region_North San Juan Mountains_20151203.nc
On Region_Grand Mesa Zone_20160218.nc
On Region_Vail Summit Zone_20160207.nc
On Region_Aspen Zone_20160218.nc
On Region_Sawatch Zone_20160218.nc
On Region_Grand Mesa Zone_20170204.nc
On Region_Gunnison Zone_20160102.nc
On Region_Vail Summit Zone_20170126.nc
On Region_Grand Mesa Zone_20160219.nc
On Region_Aspen Zone_20160219.nc
On Region_Vail Summit Zone_20160208.nc
On Region_Sangre de Cristo Range_20151129.nc
On Region_Sawatch Zone_20160219.nc
On Region_Front Range Zone_20151121.nc
On Region_Grand Mesa Zone_20170205.nc
On Region_Grand Mesa Zone_20160220.nc
On Region_

...

On Region_South San Juan Mountains_20180319.nc
On Region_Sangre de Cristo Range_20190202.nc
On Region_Gunnison Zone_20190113.nc
On Region_South San Juan Mountains_20181227.nc
On Region_North San Juan Mountains_20190104.nc
On Region_North San Juan Mountains_20180330.nc
On Region_Front Range Zone_20180316.nc
On Region_Gunnison Zone_20190114.nc
On Region_Front Range Zone_20190101.nc
On Region_South San Juan Mountains_20180320.nc
On Region_Gunnison Zone_20190115.nc
On Region_Steamboat Zone_20190118.nc
On Region_Sangre de Cristo Range_20190203.nc
On Region_South San Juan Mountains_20181228.nc
On Region_Steamboat Zone_20180410.nc
On Region_North San Juan Mountains_20190105.nc
On Region_North San Juan Mountains_20180331.nc
On Region_Gunnison Zone_20190116.nc
On Region_South San Juan Mountains_20180321.nc
On Region_Gunnison Zone_20190117.nc
On Region_Front Range Zone_20180317.nc
On Region_Front Range Zone_20190102.nc
On Region_South San Juan Mountains_20181229.nc
On Region_North San Juan Mount

On Region_North San Juan Mountains_20180419.nc
On Region_Sangre de Cristo Range_20190220.nc
On Region_Gunnison Zone_20190222.nc
On Region_South San Juan Mountains_20190116.nc
On Region_North San Juan Mountains_20190125.nc
On Region_South San Juan Mountains_20180408.nc
On Region_Gunnison Zone_20190223.nc
On Region_Front Range Zone_20190114.nc
On Region_Front Range Zone_20180329.nc
On Region_Steamboat Zone_20190202.nc
On Region_North San Juan Mountains_20180420.nc
On Region_Gunnison Zone_20190224.nc
On Region_Steamboat Zone_20180425.nc
On Region_Sangre de Cristo Range_20190221.nc
On Region_South San Juan Mountains_20190117.nc
On Region_North San Juan Mountains_20190126.nc
On Region_South San Juan Mountains_20180409.nc
On Region_Gunnison Zone_20190225.nc
On Region_North San Juan Mountains_20180421.nc
On Region_Gunnison Zone_20190226.nc
On Region_Steamboat Zone_20190203.nc
On Region_North San Juan Mountains_20190127.nc
On Region_Sangre de Cristo Range_20190222.nc
On Region_South San Juan M

On Region_North San Juan Mountains_20190217.nc
On Region_Sangre de Cristo Range_20190312.nc
On Region_South San Juan Mountains_20190207.nc
On Region_Gunnison Zone_20190408.nc
On Region_Front Range Zone_20190128.nc
On Region_South San Juan Mountains_20180430.nc
On Region_Gunnison Zone_20190409.nc
On Region_Front Range Zone_20180412.nc
On Region_North San Juan Mountains_20190218.nc
On Region_Steamboat Zone_20190219.nc
On Region_Sangre de Cristo Range_20190313.nc
On Region_South San Juan Mountains_20190208.nc
On Region_Gunnison Zone_20190410.nc
On Region_Gunnison Zone_20190411.nc
On Region_North San Juan Mountains_20190219.nc
On Region_Front Range Zone_20190129.nc
On Region_South San Juan Mountains_20190209.nc
On Region_Gunnison Zone_20190412.nc
On Region_Sangre de Cristo Range_20190314.nc
On Region_Steamboat Zone_20190220.nc
On Region_Front Range Zone_20180413.nc
On Region_Gunnison Zone_20190413.nc
On Region_North San Juan Mountains_20190220.nc
On Region_South San Juan Mountains_20190210

On Region_South San Juan Mountains_20190313.nc
On Region_Sangre de Cristo Range_20190412.nc
On Region_Front Range Zone_20190219.nc
On Region_North San Juan Mountains_20190326.nc
On Region_Steamboat Zone_20190317.nc
On Region_South San Juan Mountains_20190314.nc
On Region_Sangre de Cristo Range_20190413.nc
On Region_North San Juan Mountains_20190327.nc
On Region_Front Range Zone_20190220.nc
On Region_South San Juan Mountains_20190315.nc
On Region_Steamboat Zone_20190318.nc
On Region_Sangre de Cristo Range_20190414.nc
On Region_North San Juan Mountains_20190328.nc
On Region_South San Juan Mountains_20190316.nc
On Region_Sangre de Cristo Range_20190415.nc
On Region_Front Range Zone_20190221.nc
On Region_Steamboat Zone_20190319.nc
On Region_North San Juan Mountains_20190329.nc
On Region_South San Juan Mountains_20190317.nc
On Region_Sangre de Cristo Range_20190416.nc
On Region_North San Juan Mountains_20190330.nc
On Region_Steamboat Zone_20190320.nc
On Region_Front Range Zone_20190222.nc
O

On Region_Front Range Zone_20190403.nc
On Region_Front Range Zone_20190404.nc
On Region_Front Range Zone_20190405.nc
On Region_Front Range Zone_20190406.nc
On Region_Front Range Zone_20190407.nc
On Region_Front Range Zone_20190408.nc
On Region_Front Range Zone_20190409.nc
On Region_Front Range Zone_20190410.nc
On Region_Front Range Zone_20190411.nc
On Region_Front Range Zone_20190412.nc
On Region_Front Range Zone_20190413.nc
On Region_Front Range Zone_20190414.nc
On Region_Front Range Zone_20190415.nc
On Region_Front Range Zone_20190416.nc
On Region_Front Range Zone_20190417.nc
On Region_Front Range Zone_20190418.nc
On Region_Front Range Zone_20190419.nc
On Region_Front Range Zone_20190420.nc
On Region_Front Range Zone_20190421.nc
On Region_Front Range Zone_20190422.nc
On Region_Front Range Zone_20190423.nc
On Region_Front Range Zone_20190424.nc
On Region_Front Range Zone_20190425.nc
On Region_Front Range Zone_20190426.nc
On Region_Front Range Zone_20190427.nc
On Region_Front Range Zon